## **2020 ADS CREATION V10.4**
v10.4.2020 - Added dataset for 2020 to create validation set so that models can be tested 

v10.4.0 - Removed other event from the data-set, identified 4 major outages and removed them for outages, also used IQR: <br>
- Used IQR range 0.1 to 0.99 and removed data-set based on these ranges

v10.3.0 - Adding no of outages based on CLUE, CAUSE, OCCURN: <br>
- 'NO_OF_POWER_OUT_CLUE_PER_DAY', 'NO_OF_OPEN_DEVICE_CLUE_PER_DAY', 'NO_OF_IVR_CLUE_PER_DAY', 'NO_OF_ANIMAL_CAUSE_PER_DAY', 'NO_OF_WIRE_OCCURN_PER_DAY' <br>
- Categorized outages based on dispatch area location '34th', 'ARL.', 'MILL', 'ENGLISH', 'W.I.', 'SOUTH' and 'NO_LOCATION' based on minimum distance from outage <br>

v10.2.0 - Adding priority/queueing feature into the analytical dataset based on : <br>
- Rank based on simple customer quantity as mentioned by Eric (live rankings to be followed, numerical feature) <br>
- Rank based on the factor of distance from centroid and customer quantity (live rankings to be followed, approach #2, numerical feature) <br>

In [1]:
import csv
import math
import time
import warnings
import operator
import statistics
import seaborn as sns
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt

from dateutil.parser import parse
from datetime import datetime
from scipy import stats
from IPython.display import display_html
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

## **FILTERING FAC_JOB TABLE**

In [2]:
%%time
df_facility_job_his = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/IPL/HIS_FACILITY_JOB_IPL.csv', sep = ",", encoding = "ISO-8859-1")

CPU times: user 16.8 s, sys: 2.21 s, total: 19 s
Wall time: 29 s


In [3]:
# changing the date format

df_facility_job_his["CREATION_DATETIME"] = pd.to_datetime(df_facility_job_his["CREATION_DATETIME"], errors ='coerce')
df_facility_job_his["ENERGIZED_DATETIME"] = pd.to_datetime(df_facility_job_his["ENERGIZED_DATETIME"], errors ='coerce')
df_facility_job_his["ETR_DATETIME"] = pd.to_datetime(df_facility_job_his["ETR_DATETIME"], errors ='coerce')

# creating blue sky flags

df_facility_job_his['BLUE_SKY_FLG'] = ( (df_facility_job_his.TOT_LOSS_POWER_FLG == 'T') & 
                                           ((df_facility_job_his.MAJ_OTG_ID == 0) | (df_facility_job_his.MAJ_OTG_ID.isnull())) &
                                           ((df_facility_job_his.ISOLATED_TO_CUST_FLG == 'F') | (df_facility_job_his.ISOLATED_TO_CUST_FLG.isnull())) &
                                           ((df_facility_job_his.ROUTINE_FLG == 'F') | (df_facility_job_his.ROUTINE_FLG.isnull())) &
                                           ((df_facility_job_his.ENERGIZED_DATETIME - 
                                             df_facility_job_his.CREATION_DATETIME).dt.total_seconds().div(60).round(2) > 5) 
                                          & (df_facility_job_his.CREATION_DATETIME.dt.year > 2019) & (df_facility_job_his.CREATION_DATETIME.dt.year <= 2020))
print("Total blue sky Events: ", len(df_facility_job_his[df_facility_job_his.BLUE_SKY_FLG == True]))

# creating storm event flags

df_facility_job_his['STORM_EVENT_FLG'] = ( (df_facility_job_his.TOT_LOSS_POWER_FLG == 'T') & 
                                           ((df_facility_job_his.MAJ_OTG_ID != 0) & (df_facility_job_his.MAJ_OTG_ID.notnull())) &
                                           ((df_facility_job_his.ISOLATED_TO_CUST_FLG == 'F') | (df_facility_job_his.ISOLATED_TO_CUST_FLG.isnull())) &
                                           ((df_facility_job_his.ROUTINE_FLG == 'F') | (df_facility_job_his.ROUTINE_FLG.isnull())) &
                                           ((df_facility_job_his.ENERGIZED_DATETIME -
                                             df_facility_job_his.CREATION_DATETIME).dt.total_seconds().div(60).round(2) > 5) 
                                          & (df_facility_job_his.CREATION_DATETIME.dt.year > 2019) & (df_facility_job_his.CREATION_DATETIME.dt.year <= 2020))
print("Total Storm Events: ", len(df_facility_job_his[df_facility_job_his.STORM_EVENT_FLG == True]))


######################################################################################################################################################################################################
######################################################################### APPLYING FILTERS FOR CORRECT DATA INPUTS####################################################################################
######################################################################################################################################################################################################
print("\n")
print("After Filtering Creation_datetime from 2020 & greater")
df_facility_job_his = df_facility_job_his[(df_facility_job_his.CREATION_DATETIME.dt.year > 2019) & (df_facility_job_his.CREATION_DATETIME.dt.year <= 2020)]
_incident_ = len(df_facility_job_his[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_facility_job_his))
print("blue sky events", len(df_facility_job_his[df_facility_job_his.BLUE_SKY_FLG == True]))
print("Storm events", len(df_facility_job_his[df_facility_job_his.STORM_EVENT_FLG == True]))
print("Number of incident id", df_facility_job_his.INCIDENT_ID.nunique())
print(df_facility_job_his.shape)
print("\n")

# customer quantity greater than 0
print('Filter for customer quantity greater than 0')
# print("****QC Check****")
print("Rows left after checking for INCIDENTS whose CUSTOMER QUANTITY IS > 0")
df_facility_job_his = df_facility_job_his[(df_facility_job_his.CUST_QTY > 0)]
print(df_facility_job_his.shape)
print("\n")

# equip_stn_no is not NCC and not null
print('Filter for equp_stn_no is not NCC or not null')
# print("****QC Check****")
print("Rows left after checking that EQUIP_STN_NO is not from <<NON CONNECTED CUSTOMERS>>")
df_facility_job_his = df_facility_job_his[(df_facility_job_his.EQUIP_STN_NO != '<NCC>') & (df_facility_job_his.EQUIP_STN_NO.notnull())]
print(df_facility_job_his.shape)
print("\n")


# removing NAN from DNI_EQUIP_TYPE, CIRCT_ID, STRCTUR_NO
print('Removing NAN from DNI_EQIP_TYPE, CICRT_ID, STRCTUR_NO')
# print("****QC Check****")
print("Rows left after checking CIRCT_ID is not 0 and not null, STRCTUR_NO is not null and DNI_EQIP_TYPE is not null")
df_facility_job_his = df_facility_job_his[(df_facility_job_his.CIRCT_ID != 0)]
df_facility_job_his = df_facility_job_his[~df_facility_job_his.CIRCT_ID.isnull()]
df_facility_job_his = df_facility_job_his[~df_facility_job_his.STRCTUR_NO.isnull()]
df_facility_job_his = df_facility_job_his[~df_facility_job_his.DNI_EQUIP_TYPE.isnull()]
print(df_facility_job_his.shape)
print("\n")

# removing CLUE_CD which start with 0 but does not start with 00
print('Removing CLUE_CD which start with 0 but do not start with 00')
# print("****QC Check****")
print("Rows left after filtering for CLUE CODES which start with 0 but do not start with 00")
df_facility_job_his = df_facility_job_his[(df_facility_job_his.CLUE_CD.str[:1] == '0') & (df_facility_job_his.CLUE_CD.str[:2] != '00')]
df_facility_job_his = df_facility_job_his[df_facility_job_his.CLUE_CD != '01']
print(df_facility_job_his.shape)
print("\n")

# removing occurence codes starting with cancel, found ok and duplicate
print('Removing CLUE_CD which start with 0 but do not start with 00')
# print("****QC Check****")

print("Rows left after removing OCCURN_CD which have descriptions starting with CANCEL, FOUND OK or DUPLICATE")
occur_remov = [30003001, 33003301, 33003302, 34003400, 34003401, 34003402, 34003403, 34003404, 34003405, 34003406, 34003407, 34003408, 34003409, 35003500,
                35003501, 35003502, 35003503, 35003504, 35003505, 35003506, 35003507, 35003508, 36003600, 36003601, 36003602, 36003603, 36003604, 36003605,
                36003606, 36003607, 36003608, 37003703, 38003802, 38003803, 38003804, 38003807, 39003910, 41004100, 41004101, 41004102, 48004800, 48004802,
                48004803, 49004900, 49004901, 49004902, 50005000, 50005001, 50005002, 52005200, 52005201, 52005202, 52005203, 52005204, 52005205, 52005206,
                52005207, 53005300, 53005301, 53005302, 53005303, 53005304, 53005305, 53005306, 53005307, 53005308, 53005309, 53005310, 54005400, 54005401,
                54005402, 54005403, 54005404, 54005405, 34003410, 30003000, 36503650, 36503651, 36503652, 36503653, 36503654, 36503655, 36503656, 36503657,
                36503658]
df_facility_job_his = df_facility_job_his[~(df_facility_job_his.OCCURN_CD.isin(occur_remov))]
print(df_facility_job_his.shape)
print("\n")

Total blue sky Events:  5365
Total Storm Events:  687


After Filtering Creation_datetime from 2020 & greater
Rows 39638
blue sky events 5365
Storm events 687
Number of incident id 33207
(39638, 72)


Filter for customer quantity greater than 0
Rows left after checking for INCIDENTS whose CUSTOMER QUANTITY IS > 0
(24066, 72)


Filter for equp_stn_no is not NCC or not null
Rows left after checking that EQUIP_STN_NO is not from <<NON CONNECTED CUSTOMERS>>
(24001, 72)


Removing NAN from DNI_EQIP_TYPE, CICRT_ID, STRCTUR_NO
Rows left after checking CIRCT_ID is not 0 and not null, STRCTUR_NO is not null and DNI_EQIP_TYPE is not null
(24001, 72)


Removing CLUE_CD which start with 0 but do not start with 00
Rows left after filtering for CLUE CODES which start with 0 but do not start with 00
(15320, 72)


Removing CLUE_CD which start with 0 but do not start with 00
Rows left after removing OCCURN_CD which have descriptions starting with CANCEL, FOUND OK or DUPLICATE
(15320, 72)




In [4]:
df_fac_final = df_facility_job_his.copy(deep=True)
print("Rows", len(df_fac_final))
_incident_ = len(df_fac_final[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("blue sky events", len(df_fac_final[df_fac_final.BLUE_SKY_FLG == 1]))
print("Storm events", len(df_fac_final[df_fac_final.STORM_EVENT_FLG == 1]))
print("Number of incident id", df_fac_final.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)
print(df_fac_final.shape)

Rows 15320
blue sky events 4227
Storm events 657
Number of incident id 11577
Unique structure no 13672
(15320, 72)


In [5]:
df_event_flg = df_fac_final[['INCIDENT_ID','STRCTUR_NO','CIRCT_ID' ,'DNI_EQUIP_TYPE','STORM_EVENT_FLG']]
def event_flag(group):
    group = group.reset_index(drop = True)
    if(group.STORM_EVENT_FLG.sum() >=1):
        group['EVENT'] = 'STORM'
        return group
    else:
        group['EVENT'] = 'BLUE SKY'
        return group
df_event_flg = df_event_flg.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).apply(event_flag).reset_index(drop = True)

In [6]:
df_check = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE']).nunique()
df_check.sum()

FAC_JOB_ID              15320
MAJ_OTG_ID              13192
EQUIP_STN_NO            15253
DIST_NO                 13676
HOST_SEQ_ID                 0
PRIORITY_VAL            13858
CUST_QTY                15064
CLUE_CD                 14275
CLUE_DESC               14275
CREATION_DATETIME       13773
CALL_QTY                14903
KEY_CUST_QTY            14282
SPLIT_FAC_JOB_FLG        8059
CAUSE_CD                 6971
CAUSE_DESC               6906
OCCURN_CD               13669
OCCURN_DESC             13296
CLIMATIC_CD             13664
CLIMATIC_DESC           13664
CITY_NAM                13374
LOC_DESC                13613
WRK_ORD_NUM                 0
COMMENT_TEXT             6567
CALL_ID                 15320
KVA_VAL                 14868
BOOK_NO                     0
ADDRESS                 14798
CIRCT_NAM               13676
CLUE_CD2                  298
INSERTED_DATE           13712
DOWNSTREAM_KVA_VAL      14946
DOWNSTREAM_CUST_QTY     15064
COMPL_DATETIME          13757
TOT_LOSS_P

In [7]:
df_numerical = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE' ], as_index = False).agg({'CUST_QTY':'sum','CALL_QTY':'sum','KEY_CUST_QTY':'sum',
                                                                                                                        'DOWNSTREAM_CUST_QTY':'sum','KVA_VAL':'mean',
                                                                                                                        'DOWNSTREAM_KVA_VAL':'mean', 'FAC_JOB_ID': 'max',
                                                                                                                        'ETR_DATETIME': 'max', 'CREATION_DATETIME': 'min',
                                                                                                                        'MAJ_OTG_ID' : 'max','ENERGIZED_DATETIME': 'max',
                                                                                                                        'SUBST_ID': 'min', 'COMMENT_TEXT' : 'last'})

In [8]:
print(df_numerical.shape)
display(df_numerical.head())

(13676, 17)


,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,CUST_QTY,CALL_QTY,KEY_CUST_QTY,DOWNSTREAM_CUST_QTY,KVA_VAL,DOWNSTREAM_KVA_VAL,FAC_JOB_ID,ETR_DATETIME,CREATION_DATETIME,MAJ_OTG_ID,ENERGIZED_DATETIME,SUBST_ID,COMMENT_TEXT
0,2001465208.00,462-B/97,3209.00,FUSE,30.00,26,1.00,30.00,100.00,100.00,2002645939,2020-01-01 03:45:00,2020-01-01 00:03:00,0.00,2020-01-01 02:30:00,320.00,NaN
1,2001465216.00,481CB/195,2902.00,1TBOH,1.00,1,0.00,1.00,0.00,0.00,2002645942,2020-01-01 04:30:00,2020-01-01 00:50:44,0.00,2020-01-01 01:15:40,290.00,NaN
2,2001465223.00,280-B/81,3607.00,SWITCH,205.00,70,0.00,205.00,2480.70,2480.70,2002645993,2020-01-01 05:45:00,2020-01-01 03:10:16,nan,2020-01-01 04:18:28,360.00,NaN
3,2001465223.00,281-B/4,3607.00,SWITCH,10.00,3,1.00,10.00,152.00,152.00,2002645992,2020-01-01 08:00:00,2020-01-01 03:10:16,0.00,2020-01-01 05:23:41,360.00,NaN
4,2001465252.00,318--/116,3607.00,1TBOH,4.00,3,0.00,4.00,25.00,25.00,2002645982,2020-01-01 07:00:00,2020-01-01 03:23:08,0.00,2020-01-01 05:09:12,360.00,NaN


In [9]:
df_numerical = pd.merge(df_numerical, df_event_flg, on = ['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE' ], how='left')

In [10]:
df_numerical.drop_duplicates(keep='first', inplace=True)
print(df_numerical.shape)
display(df_numerical.head())

(13699, 19)


,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,CUST_QTY,CALL_QTY,KEY_CUST_QTY,DOWNSTREAM_CUST_QTY,KVA_VAL,DOWNSTREAM_KVA_VAL,FAC_JOB_ID,ETR_DATETIME,CREATION_DATETIME,MAJ_OTG_ID,ENERGIZED_DATETIME,SUBST_ID,COMMENT_TEXT,STORM_EVENT_FLG,EVENT
0,2001465208.00,462-B/97,3209.00,FUSE,30.00,26,1.00,30.00,100.00,100.00,2002645939,2020-01-01 03:45:00,2020-01-01 00:03:00,0.00,2020-01-01 02:30:00,320.00,NaN,False,BLUE SKY
1,2001465216.00,481CB/195,2902.00,1TBOH,1.00,1,0.00,1.00,0.00,0.00,2002645942,2020-01-01 04:30:00,2020-01-01 00:50:44,0.00,2020-01-01 01:15:40,290.00,NaN,False,BLUE SKY
2,2001465223.00,280-B/81,3607.00,SWITCH,205.00,70,0.00,205.00,2480.70,2480.70,2002645993,2020-01-01 05:45:00,2020-01-01 03:10:16,nan,2020-01-01 04:18:28,360.00,NaN,False,BLUE SKY
3,2001465223.00,281-B/4,3607.00,SWITCH,10.00,3,1.00,10.00,152.00,152.00,2002645992,2020-01-01 08:00:00,2020-01-01 03:10:16,0.00,2020-01-01 05:23:41,360.00,NaN,False,BLUE SKY
4,2001465252.00,318--/116,3607.00,1TBOH,4.00,3,0.00,4.00,25.00,25.00,2002645982,2020-01-01 07:00:00,2020-01-01 03:23:08,0.00,2020-01-01 05:09:12,360.00,NaN,False,BLUE SKY


## **Adding extra columns like Day Night flag and creating dependent variable TTR column**

In [11]:
# creating day night flag for outages

df_numerical['DAY_FLAG'] = df_numerical.CREATION_DATETIME.dt.hour.apply(lambda x: 1 if ((x >= 6) & (x<18)) else 0)
df_numerical['TTR'] = (df_numerical.ENERGIZED_DATETIME - df_numerical.CREATION_DATETIME).dt.total_seconds().div(60).round(4)

In [12]:
print(df_numerical.shape)

(13699, 21)


In [13]:
# df_numerical = df_numerical[(df_numerical['TTR'] >= 30)]
df_numerical['TTR'].head()
df_numerical.reset_index(drop=True, inplace=True)
print(df_numerical.shape)

(13699, 21)


In [14]:
df_fac_final = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index=False).agg({'PRIORITY_VAL': 'last', 'OCCURN_DESC' : 'last',
                                                                                                                     'CAUSE_DESC': 'last', 'CLUE_DESC': 'last', 'CITY_NAM' : 'last'})
df_fac_final.head()

,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,PRIORITY_VAL,OCCURN_DESC,CAUSE_DESC,CLUE_DESC,CITY_NAM
0,2001465208.00,462-B/97,3209.00,FUSE,2,"FUSE\OPEN, FUSE BLOWN",PUBLIC\VANDALISM,IVR\POWER OUT,INDIANAPOLIS
1,2001465216.00,481CB/195,2902.00,1TBOH,2,SERVICE\OTHER,PUBLIC\OTHER,IVR\POWER OUT,INDIANAPOLIS
2,2001465223.00,280-B/81,3607.00,SWITCH,2,CONDUCTOR/WIRE\PRIMARY DOWN,PUBLIC\POLE HIT,WEB\POWER OUT,INDIANAPOLIS
3,2001465223.00,281-B/4,3607.00,SWITCH,2,CONDUCTOR/WIRE\PRIMARY DOWN,PUBLIC\POLE HIT,WEB\POWER OUT,INDIANAPOLIS
4,2001465252.00,318--/116,3607.00,1TBOH,2,CANCEL\BY SDO,NaN,WEB\POWER OUT,INDIANAPOLIS


## **CLUE CODE CLEAN**

In [15]:
df_fac_final.PRIORITY_VAL = df_fac_final.PRIORITY_VAL.astype(float)

df_fac_final['PRIORITY_VAL_1.0'] = df_fac_final['PRIORITY_VAL'].apply(lambda x: 1 if x == 1 else 0)
df_fac_final['PRIORITY_VAL_2.0'] = df_fac_final['PRIORITY_VAL'].apply(lambda x: 1 if x == 2 else 0)
df_fac_final['PRIORITY_VAL_3.0'] = df_fac_final['PRIORITY_VAL'].apply(lambda x: 1 if x == 3 else 0)
df_fac_final['PRIORITY_VAL_5.0'] = df_fac_final['PRIORITY_VAL'].apply(lambda x: 1 if x == 5 else 0)

df_fac_final.drop(['PRIORITY_VAL'],axis=1,inplace=True)

df_fac_final.CITY_NAM = df_fac_final.CITY_NAM.apply(lambda x: 'INDIANAPOLIS' if(str(x).find('INDIAN') != -1) else x)
df_fac_final.CITY_NAM = df_fac_final.CITY_NAM.apply(lambda x: 'NO_CITY' if(x != x) else x)

df_fac_final['OCCURN_DESC'] = df_fac_final['OCCURN_DESC'].astype('str')

In [16]:
# segregation of clue code desc

df_fac_final['POLE_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('pole') != -1) else 0)
df_fac_final['PART_LIGHT_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('part lights') != -1) else 0)
df_fac_final['EMERGENCY_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('emergency') != -1) else 0)
df_fac_final['POWER_OUT_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('power out') != -1) else 0)
df_fac_final['TREE_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('tree') != -1) else 0)
df_fac_final['WIRE_DOWN_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('wire down') != -1) else 0)
df_fac_final['IVR_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('ivr') != -1) else 0)
df_fac_final['EQUIPMENT_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.find('EQUIPMENT') != -1) else 0)
df_fac_final['TRANSFORMER_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.find('TRANSFORMER') != -1) else 0)
df_fac_final['OPEN_DEVICE_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.find('OPEN DEVICE') != -1) else 0)


#segration of cause desc
df_fac_final['CAUSE_DESC1'] = df_fac_final[['CAUSE_DESC']].fillna('0')
df_fac_final['OH_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if((x.find('OH') != -1) | (x.find('O.H.') != -1)) else 0)
df_fac_final['UG_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if((x.find('UG') != -1) | (x.find('U.G.') != -1)) else 0)
df_fac_final['ANIMAL_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('ANIMAL') != -1) else 0)
df_fac_final['WEATHER_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('WEATHER') != -1) else 0)
df_fac_final['WEATHER_COLD_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('COLD') != -1) else 0)
df_fac_final['WEATHER_LIGHTNING_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('LIGHTNING') != -1) else 0)
df_fac_final['WEATHER__SNOW_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('SNOW') != -1) else 0)
df_fac_final['WEATHER__WIND_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('WIND') != -1) else 0)
df_fac_final['WEATHER__HEAT_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('HEAT') != -1) else 0)
df_fac_final['WEATHER__FLOOD_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('FLOOD') != -1) else 0)
df_fac_final['PUBLIC_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('PUBLIC') != -1) else 0)
df_fac_final['STREET_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('ST ') != -1) else 0)
df_fac_final['SUBSTATION_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('SUBSTATION') != -1) else 0)
df_fac_final['TREE_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('TREE') != -1) else 0)
df_fac_final['MISCELLANEOUS_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('MISCELLANEOUS') != -1) else 0)
df_fac_final['CUST_REQUEST_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('CUSTOMER REQUEST') != -1) else 0)
df_fac_final['NO_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('NO CAUSE') != -1) else 0)
df_fac_final['PLANNED_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('PLANNED WORK') != -1) else 0)
df_fac_final['NO_OUTAGE_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('NO OUTAGE') != -1) else 0)


#segration of OCCURN desc
df_fac_final['FUSE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if((x.find('FUSE') != -1) & (x.find('FUSE NOT') == -1)) else 0)
df_fac_final['CUST_EQUIP_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CUSTOMER EQUIP') != -1) else 0)
df_fac_final['POLE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('POLE') != -1) else 0)
df_fac_final['TRANSFORMER_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('TRANSFORMER') != -1) else 0)
df_fac_final['METER_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('METER') != -1) else 0)
df_fac_final['SERVICE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('SERVICE') != -1) else 0)
df_fac_final['CABLE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CABLE') != -1) else 0)
df_fac_final['ST_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('ST') != -1) else 0)
df_fac_final['FIRE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('FIRE') != -1) else 0)
df_fac_final['FOUND_OPEN_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if((x.find('FOUND OPEN') != -1) & (x.find('NOT FOUND OPEN') == -1)) else 0)
df_fac_final['PUBLIC_SAFETY_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('SAFETY') != -1) else 0)
df_fac_final['WIRE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('WIRE') != -1) else 0)
df_fac_final['SWITCH_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('SWITCH') != -1) else 0)
df_fac_final['CUTOUT_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CUTOUT') != -1) else 0)
df_fac_final['REGULATOR_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('REGULATOR') != -1) else 0)
df_fac_final['CAP_BANK_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CAP BANK') != -1) else 0)
df_fac_final['OH_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('OH') != -1) else 0)
df_fac_final['RECLOSER_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('RECLOSER') != -1) else 0)

df_fac_final = df_fac_final.drop(columns = ['CAUSE_DESC1'])

In [17]:
df_fac_cat = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).agg({'POLE_CLUE_FLG': 'sum', 'PART_LIGHT_CLUE_FLG': 'sum',
                                                                          'EMERGENCY_CLUE_FLG': 'sum','POWER_OUT_CLUE_FLG': 'sum',
                                                                          'TREE_CLUE_FLG': 'sum', 'WIRE_DOWN_CLUE_FLG': 'sum',
                                                                          'OPEN_DEVICE_CLUE_FLG':'sum', 'EQUIPMENT_CLUE_FLG': 'sum',
                                                                          'TRANSFORMER_CLUE_FLG':'sum','IVR_CLUE_FLG': 'sum',
                                                                          'OH_CAUSE_FLG': 'sum', 'UG_CAUSE_FLG': 'sum', 
                                                                          'ANIMAL_CAUSE_FLG': 'sum','WEATHER_CAUSE_FLG': 'sum', 
                                                                          'WEATHER_COLD_CAUSE_FLG': 'sum','PUBLIC_CAUSE_FLG': 'sum',
                                                                         'WEATHER_LIGHTNING_CAUSE_FLG': 'sum', 'WEATHER__SNOW_CAUSE_FLG': 'sum',
                                                                          'WEATHER__WIND_CAUSE_FLG': 'sum','WEATHER__HEAT_CAUSE_FLG': 'sum',
                                                                         'WEATHER__FLOOD_CAUSE_FLG': 'sum', 'STREET_CAUSE_FLG': 'sum',
                                                                        'MISCELLANEOUS_CAUSE_FLG':'sum', 'CUST_REQUEST_CAUSE_FLG': 'sum',
                                                                          'SUBSTATION_CAUSE_FLG': 'sum','TREE_CAUSE_FLG': 'sum',
                                                                          'NO_CAUSE_FLG': 'sum', 'PLANNED_CAUSE_FLG': 'sum',
                                                                          'NO_OUTAGE_CAUSE_FLG': 'sum',
                                                                          'PRIORITY_VAL_1.0' : 'sum', 'PRIORITY_VAL_2.0': 'sum', 
                                                                          'PRIORITY_VAL_3.0': 'sum', 'PRIORITY_VAL_5.0': 'sum',
                                                                          'FUSE_OCCURN_FLG': 'sum', 'CUST_EQUIP_OCCURN_FLG': 'sum',
                                                                          'POLE_OCCURN_FLG': 'sum', 'TRANSFORMER_OCCURN_FLG': 'sum', 
                                                                          'METER_OCCURN_FLG': 'sum', 'SERVICE_OCCURN_FLG': 'sum',
                                                                          'CABLE_OCCURN_FLG': 'sum', 'ST_OCCURN_FLG': 'sum',
                                                                          'FIRE_OCCURN_FLG': 'sum', 'FOUND_OPEN_OCCURN_FLG': 'sum',
                                                                          'PUBLIC_SAFETY_OCCURN_FLG': 'sum', 'WIRE_OCCURN_FLG': 'sum',
                                                                          'SWITCH_OCCURN_FLG': 'sum', 'REGULATOR_OCCURN_FLG': 'sum',
                                                                          'CUTOUT_OCCURN_FLG': 'sum','CAP_BANK_OCCURN_FLG': 'sum',
                                                                          'RECLOSER_OCCURN_FLG': 'sum','OH_OCCURN_FLG': 'sum'
                                                                          })
dummy_col = ['POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
             'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0']
for i in dummy_col:
    df_fac_cat[i] =  df_fac_cat[i].apply(lambda x: 1 if x>=1 else 0)

df_fac_cat = df_fac_cat[['INCIDENT_ID','STRCTUR_NO','CIRCT_ID', 'DNI_EQUIP_TYPE',
                         'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
              'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0']].drop_duplicates()

In [18]:
print(df_fac_cat.shape)

(13676, 55)


### **CITY**

In [19]:
%%time

# city treatment
def cat_city_treat(group):
    if(group.CITY_NAM.nunique() > 1):
        x = group[group.CITY_NAM != 'NO_CITY'].CITY_NAM.unique()
        group.CITY_NAM = x[0]
        return group
    else:
        return group
df_treated = df_fac_final[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'CITY_NAM']]
df_treated = df_treated.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).apply(cat_city_treat)

CPU times: user 9.21 s, sys: 0 ns, total: 9.21 s
Wall time: 9.2 s


In [20]:
len(df_treated[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'CITY_NAM']].drop_duplicates())

13676

QC check complete

In [21]:
df_treated = df_treated[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE','CITY_NAM']].drop_duplicates()
df_fac_cat = pd.merge(df_fac_cat, df_treated, on = ['INCIDENT_ID','STRCTUR_NO','CIRCT_ID', 'DNI_EQUIP_TYPE'])

In [22]:
print(df_fac_cat.shape)

(13676, 56)


In [23]:
df_fac_cat.head()

,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM
0,2001465208.00,462-B/97,3209.00,FUSE,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS
1,2001465216.00,481CB/195,2902.00,1TBOH,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS
2,2001465223.00,280-B/81,3607.00,SWITCH,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS
3,2001465223.00,281-B/4,3607.00,SWITCH,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS
4,2001465252.00,318--/116,3607.00,1TBOH,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS


## **X Y Coordinate treatment**

In [24]:
%%time
df_his_location = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/IPL/HIS_LOCATION_IPL.csv', sep = "|")

CPU times: user 48.8 s, sys: 7.06 s, total: 55.8 s
Wall time: 1min 24s


In [25]:
df_his_location = df_his_location[['INCIDENT_ID','STRCTUR_NO','GEO_X_COORD','GEO_Y_COORD','PRIMARY_LOC_FLG']].drop_duplicates()
df_his_location = df_his_location[df_his_location.INCIDENT_ID.isin(df_fac_final.INCIDENT_ID)]
df_his_location = df_his_location[df_his_location.STRCTUR_NO.isin(df_fac_final.STRCTUR_NO)]

### **Changing X Y coordinates to LAT and LONG**

In [26]:
%%time
# function to convert geo_x, geo_y coordinate to lat, long

def change_to_loc(df):
    demnorthing = df.GEO_Y_COORD
    demeasting = df.GEO_X_COORD
    northing = float(demnorthing) * 0.3048
    easting = float(demeasting) * 0.3048
    om = (northing - 250000 + 4151863.7425) / 6367236.89768
    fo = om + (math.sin(om) * math.cos(om)) * (0.005022893948 + 0.000029370625 * math.pow(math.cos(om), 2) + 0.000000235059 * math.pow(math.cos(om), 4) + 0.000000002181 * math.pow(math.cos(om), 6))
    tf = math.sin(fo) / math.cos(fo)
    nf2 = 0.00673949677548 * math.pow(math.cos(fo), 2)
    rn = 0.9999666667 * 6378137 / math.pow((1 - 0.0066943800229034 * math.pow(math.sin(fo), 2)), 0.5)
    q = (easting - 100000) / rn
    b2 = -0.5 * tf * (1 + nf2)
    b4 = -(1 / 12) * (5 + (3 * math.pow(tf, 2)) + (nf2 * (1 - 9 * math.pow(tf, 2)) - 4 * math.pow(nf2, 2)))
    b6 = (1 / 360) * (61 + (90 * math.pow(tf, 2)) + (45 * math.pow(tf, 4)) + (nf2 * (46 - (252 * math.pow(tf, 2)) - (90 * math.pow(tf, 4)))))
    lat = fo + b2 * math.pow(q, 2) * (1 + math.pow(q, 2) * (b4 + b6 * math.pow(q, 2)))
    b3 = -(1 / 6) * (1 + 2 * math.pow(tf, 2) + nf2)
    b5 = (1 / 120) * (5 + 28 * math.pow(tf, 2) + 24 * math.pow(tf, 4) + nf2 * (6 + 8 * math.pow(tf, 2)))
    b7 = -(1 / 5040) * (61 + 662 * math.pow(tf, 2) + 1320 * math.pow(tf, 4) + 720 * math.pow(tf, 6))
    l = q * (1 + math.pow(q, 2) * (b3 + math.pow(q, 2) * (b5 + b7 * math.pow(q, 2))))
    lon = 1.4951653925 - l / math.cos(fo)
    coord = [(lat * 57.2957795131), (-1 * lon * 57.2957795131)]
    return coord[0],coord[1]

df_his_location['LAT'],df_his_location['LONG'] = zip(*df_his_location.apply(change_to_loc, axis=1))

CPU times: user 355 ms, sys: 0 ns, total: 355 ms
Wall time: 353 ms


In [27]:
(df_his_location['INCIDENT_ID'].astype(str) + df_his_location['STRCTUR_NO'].astype(str)).nunique(), len(df_his_location)

(12809, 12911)

Multiple locations were present, so had to do treating

In [ ]:
%%time
def loc_treatment(group):
    group = group.reset_index(drop = True)
    if((group.LAT.nunique() > 1) | (group.LONG.nunique() > 1)):
        x = 0.0
        y = 0.0
        z = 0.0
        for i in range(len(group)):
            latitude = math.radians(float(group.LAT[i]))
            longitude = math.radians(float(group.LONG[i]))
            x += math.cos(latitude) * math.cos(longitude)
            y += math.cos(latitude) * math.sin(longitude)
            z += math.sin(latitude)
        total = len(group)
        x = x / total
        y = y / total
        z = z / total
        central_longitude = math.atan2(y, x)
        central_square_root = math.sqrt(x * x + y * y)
        central_latitude = math.atan2(z, central_square_root)
        group.LAT = math.degrees(central_latitude)
        group.LONG = math.degrees(central_longitude)
        return group
    else:
        return group

df_his_location = df_his_location.groupby(['INCIDENT_ID','STRCTUR_NO'], as_index = False).apply(loc_treatment)

CPU times: user 26.4 s, sys: 6.73 s, total: 33.1 s
Wall time: 21.5 s


In [29]:
df_his_location = df_his_location[['INCIDENT_ID','STRCTUR_NO','LAT','LONG']].drop_duplicates()

In [30]:
(df_his_location['INCIDENT_ID'].astype(str) + df_his_location['STRCTUR_NO'].astype(str)).nunique(), len(df_his_location)

(12809, 12809)

In [31]:
df_his_location['LAT'] = df_his_location['LAT'].astype(float)
df_his_location['LONG'] = df_his_location['LONG'].astype(float)
df_his_location['LAT'].min(), df_his_location['LAT'].max()

(39.4502983632741, 39.981755831883206)

In [32]:
df_his_location['LONG'].min(), df_his_location['LONG'].max()

(-86.7218249072067, -85.91821616389558)

In [33]:
# function to add zone feature to the ads according to geo coordinates
def add_zone_feature(df):
    center_lat = 39.7684
    center_long = -86.1581
    zone = ''
    
    if(float(df['LAT']) < center_lat):
        if(float(df['LONG']) < center_long):
            zone = 'ZONE1'
        else:
            zone = 'ZONE2'
    else:
        if(float(df['LONG']) < center_long):
            zone = 'ZONE4'
        else:
            zone = 'ZONE3'
    
    return zone

df_his_location['ZONE'] = df_his_location.apply(add_zone_feature, axis=1)
print(df_his_location['ZONE'].unique())

['ZONE4' 'ZONE3' 'ZONE1' 'ZONE2']


In [34]:
df_fac_cat = pd.merge(df_fac_cat, df_his_location, on = ['INCIDENT_ID','STRCTUR_NO'], how = "left")

In [35]:
print(df_fac_cat.shape)
df_fac_cat.head()

(13676, 59)


,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE
0,2001465208.00,462-B/97,3209.00,FUSE,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.79,-86.19,ZONE4
1,2001465216.00,481CB/195,2902.00,1TBOH,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.80,-85.99,ZONE3
2,2001465223.00,280-B/81,3607.00,SWITCH,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3
3,2001465223.00,281-B/4,3607.00,SWITCH,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.01,ZONE3
4,2001465252.00,318--/116,3607.00,1TBOH,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.87,-86.04,ZONE3


In [36]:
df_ads = pd.merge(df_numerical, df_fac_cat, on = ['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], how = "inner")

In [37]:
print(df_fac_cat.shape)
print(df_numerical.shape)
print(df_ads.shape)
df_ads.head()

(13676, 59)
(13699, 21)
(13699, 76)


,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,CUST_QTY,CALL_QTY,KEY_CUST_QTY,DOWNSTREAM_CUST_QTY,KVA_VAL,DOWNSTREAM_KVA_VAL,FAC_JOB_ID,ETR_DATETIME,CREATION_DATETIME,MAJ_OTG_ID,ENERGIZED_DATETIME,SUBST_ID,COMMENT_TEXT,STORM_EVENT_FLG,EVENT,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE
0,2001465208.00,462-B/97,3209.00,FUSE,30.00,26,1.00,30.00,100.00,100.00,2002645939,2020-01-01 03:45:00,2020-01-01 00:03:00,0.00,2020-01-01 02:30:00,320.00,NaN,False,BLUE SKY,0,147.00,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.79,-86.19,ZONE4
1,2001465216.00,481CB/195,2902.00,1TBOH,1.00,1,0.00,1.00,0.00,0.00,2002645942,2020-01-01 04:30:00,2020-01-01 00:50:44,0.00,2020-01-01 01:15:40,290.00,NaN,False,BLUE SKY,0,24.93,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.80,-85.99,ZONE3
2,2001465223.00,280-B/81,3607.00,SWITCH,205.00,70,0.00,205.00,2480.70,2480.70,2002645993,2020-01-01 05:45:00,2020-01-01 03:10:16,nan,2020-01-01 04:18:28,360.00,NaN,False,BLUE SKY,0,68.20,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3
3,2001465223.00,281-B/4,3607.00,SWITCH,10.00,3,1.00,10.00,152.00,152.00,2002645992,2020-01-01 08:00:00,2020-01-01 03:10:16,0.00,2020-01-01 05:23:41,360.00,NaN,False,BLUE SKY,0,133.42,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.01,ZONE3
4,2001465252.00,318--/116,3607.00,1TBOH,4.00,3,0.00,4.00,25.00,25.00,2002645982,2020-01-01 07:00:00,2020-01-01 03:23:08,0.00,2020-01-01 05:09:12,360.00,NaN,False,BLUE SKY,0,106.07,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.87,-86.04,ZONE3


## **ADD NO OF OUTAGES FOR CLUE, CAUSE, OCCURN**

In [38]:
df_ads['CREATION_DATETIME'] = pd.to_datetime(df_ads['CREATION_DATETIME'])
df_ads['Date'] = df_ads['CREATION_DATETIME'].dt.date

df_no_of_outages = df_ads.groupby(['Date'],as_index=False).agg({'POWER_OUT_CLUE_FLG' : 'sum', 'OPEN_DEVICE_CLUE_FLG' : 'sum', 'IVR_CLUE_FLG' : 'sum', 'ANIMAL_CAUSE_FLG' : 'sum',
                                                                'WIRE_OCCURN_FLG' : 'sum'})
df_no_of_outages.rename(columns = {'POWER_OUT_CLUE_FLG' : 'NO_OF_POWER_OUT_CLUE_PER_DAY', 'OPEN_DEVICE_CLUE_FLG' : 'NO_OF_OPEN_DEVICE_CLUE_PER_DAY',
                                   'IVR_CLUE_FLG' : 'NO_OF_IVR_CLUE_PER_DAY', 'ANIMAL_CAUSE_FLG' : 'NO_OF_ANIMAL_CAUSE_PER_DAY',
                                   'WIRE_OCCURN_FLG' : 'NO_OF_WIRE_OCCURN_PER_DAY'}, inplace=True)

df_no_of_outages.head()

,Date,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY
0,2020-01-01,37,2,17,8,5
1,2020-01-02,64,3,26,17,7
2,2020-01-03,43,1,21,4,2
3,2020-01-04,22,0,11,1,1
4,2020-01-05,19,5,8,4,3


In [40]:
df_ads = df_ads[['FAC_JOB_ID','INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 
         'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 
         'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 
                 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
              'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0', 'CITY_NAM', 'LAT', 'LONG', 'ZONE', 'Date', 'COMMENT_TEXT']]

df_ads.columns = ['OUTAGE_ID','INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 
         'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 
         'KVA_VAL', 'DAY_FLAG', 'TTR',
                  'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
              'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0', 'CITY_NAM', 'LAT', 'LONG','ZONE', 'Date', 'COMMENT_TEXT']

In [41]:
df_ads = pd.merge(df_ads, df_no_of_outages, on=['Date'], how='left')

In [42]:
df_ads.drop(['Date'],axis=1,inplace=True)

In [43]:
print(df_ads.shape)
df_ads.head()

(13699, 79)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY
0,2002645939,2001465208.00,462-B/97,BLUE SKY,2020-01-01 00:03:00,2020-01-01 02:30:00,3209.00,FUSE,320.00,26,30.00,1.00,2020-01-01 03:45:00,30.00,100.00,100.00,0,147.00,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.79,-86.19,ZONE4,NaN,37,2,17,8,5
1,2002645942,2001465216.00,481CB/195,BLUE SKY,2020-01-01 00:50:44,2020-01-01 01:15:40,2902.00,1TBOH,290.00,1,1.00,0.00,2020-01-01 04:30:00,1.00,0.00,0.00,0,24.93,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.80,-85.99,ZONE3,NaN,37,2,17,8,5
2,2002645993,2001465223.00,280-B/81,BLUE SKY,2020-01-01 03:10:16,2020-01-01 04:18:28,3607.00,SWITCH,360.00,70,205.00,0.00,2020-01-01 05:45:00,205.00,2480.70,2480.70,0,68.20,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3,NaN,37,2,17,8,5
3,2002645992,2001465223.00,281-B/4,BLUE SKY,2020-01-01 03:10:16,2020-01-01 05:23:41,3607.00,SWITCH,360.00,3,10.00,1.00,2020-01-01 08:00:00,10.00,152.00,152.00,0,133.42,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.01,ZONE3,NaN,37,2,17,8,5
4,2002645982,2001465252.00,318--/116,BLUE SKY,2020-01-01 03:23:08,2020-01-01 05:09:12,3607.00,1TBOH,360.00,3,4.00,0.00,2020-01-01 07:00:00,4.00,25.00,25.00,0,106.07,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.87,-86.04,ZONE3,NaN,37,2,17,8,5


In [44]:
ads = df_ads.copy(deep=True)
ads=ads[['OUTAGE_ID', 'LAT', 'LONG']]
ads.reset_index(drop=True,inplace=True)
ads.head()

,OUTAGE_ID,LAT,LONG
0,2002645939,39.79,-86.19
1,2002645942,39.80,-85.99
2,2002645993,39.88,-86.02
3,2002645992,39.88,-86.01
4,2002645982,39.87,-86.04


In [45]:
ads['LAT'] = pd.to_numeric(ads['LAT'])
ads['LONG'] = pd.to_numeric(ads['LONG'])
print(ads.dtypes)
ads.head()

OUTAGE_ID      int64
LAT          float64
LONG         float64
dtype: object


,OUTAGE_ID,LAT,LONG
0,2002645939,39.79,-86.19
1,2002645942,39.80,-85.99
2,2002645993,39.88,-86.02
3,2002645992,39.88,-86.01
4,2002645982,39.87,-86.04


In [46]:
ads['Marker1_LAT'] =  39.9613 
ads['Marker2_LAT'] = 39.8971
ads['Marker3_LAT'] = 39.9060
ads['Marker4_LAT'] = 39.9024
ads['Marker5_LAT'] = 39.8960
ads['Marker6_LAT'] = 39.8339
ads['Marker7_LAT'] = 39.8412
ads['Marker8_LAT'] = 39.8381
ads['Marker9_LAT'] = 39.8386
ads['Marker10_LAT'] = 39.7579
ads['Marker11_LAT'] = 39.7621
ads['Marker12_LAT'] = 39.7621
ads['Marker13_LAT'] = 39.7695
ads['Marker14_LAT'] = 39.6617
ads['Marker15_LAT'] = 39.6639
ads['Marker16_LAT'] = 39.6702
ads['Marker17_LAT'] = 39.6744
ads['Marker18_LAT'] = 39.5909
ads['Marker19_LAT'] = 39.5295
ads['Marker20_LAT'] = 39.5475

ads['Marker1_LONG'] = -86.4034 
ads['Marker2_LONG'] = -86.3045
ads['Marker3_LONG'] = -86.2001
ads['Marker4_LONG'] = -86.0738
ads['Marker5_LONG'] = -85.9783
ads['Marker6_LONG'] = -86.3155
ads['Marker7_LONG'] = -86.2056
ads['Marker8_LONG'] = -86.0985
ads['Marker9_LONG'] = -85.9811
ads['Marker10_LONG'] = -86.3155
ads['Marker11_LONG'] = -86.2042
ads['Marker12_LONG'] = -86.0923
ads['Marker13_LONG'] = -85.9708
ads['Marker14_LONG'] = -86.2935
ads['Marker15_LONG'] = -86.1823
ads['Marker16_LONG'] = -86.0669
ads['Marker17_LONG'] = -85.9557
ads['Marker18_LONG'] = -86.4212
ads['Marker19_LONG'] = -86.5874
ads['Marker20_LONG'] = -86.2743

In [47]:
# calculate distance from 2 lat long 

def haversine(p1, p2):
    R = 6371     # earth radius in km
    p1 = [math.radians(v) for v in p1]
    p2 = [math.radians(v) for v in p2]

    d_lat = p2[0] - p1[0]
    d_lng = p2[1] - p1[1]
    a = math.pow(math.sin(d_lat / 2), 2) + math.cos(p1[0]) * math.cos(p2[0]) * math.pow(math.sin(d_lng / 2), 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c   # returns distance between p1 and p2 in km


In [48]:
# calculate minimum distance

def minimum_distance(lat, long, marker1_lat, marker2_lat, marker3_lat, marker4_lat, marker5_lat, marker6_lat, marker7_lat, marker8_lat, marker9_lat, marker10_lat, marker11_lat,
                     marker12_lat, marker13_lat, marker14_lat, marker15_lat, marker16_lat, marker17_lat, marker18_lat, marker19_lat, marker20_lat, marker1_long, marker2_long,
                     marker3_long, marker4_long, marker5_long, marker6_long, marker7_long, marker8_long, marker9_long, marker10_long, marker11_long, marker12_long, marker13_long,
                     marker14_long, marker15_long, marker16_long, marker17_long, marker18_long, marker19_long, marker20_long):
    
    dist1 = haversine((lat,long), (marker1_lat, marker1_long))
    dist2 = haversine((lat,long), (marker2_lat, marker2_long))
    dist3 = haversine((lat,long), (marker3_lat, marker3_long))
    dist4 = haversine((lat,long), (marker4_lat, marker4_long))
    dist5 = haversine((lat,long), (marker5_lat, marker5_long))
    dist6 = haversine((lat,long), (marker6_lat, marker6_long))
    dist7 = haversine((lat,long), (marker7_lat, marker7_long))
    dist8 = haversine((lat,long), (marker8_lat, marker8_long))
    dist9 = haversine((lat,long), (marker9_lat, marker9_long))
    dist10 = haversine((lat,long), (marker10_lat, marker10_long))
    dist11 = haversine((lat,long), (marker11_lat, marker11_long))
    dist12 = haversine((lat,long), (marker12_lat, marker12_long))
    dist13 = haversine((lat,long), (marker13_lat, marker13_long))
    dist14 = haversine((lat,long), (marker14_lat, marker14_long))
    dist15 = haversine((lat,long), (marker15_lat, marker15_long))
    dist16 = haversine((lat,long), (marker16_lat, marker16_long))
    dist17 = haversine((lat,long), (marker17_lat, marker17_long))
    dist18 = haversine((lat,long), (marker18_lat, marker18_long))
    dist19 = haversine((lat,long), (marker19_lat, marker19_long))
    dist20 = haversine((lat,long), (marker20_lat, marker20_long))
    
    dist_list = [dist1, dist2, dist3, dist4, dist5, dist6, dist7, dist8, dist9, dist10, dist11, dist12, dist13, dist14, dist15, dist16, dist17, dist18, dist19, dist20]

    min_index, min_value = min(enumerate(dist_list), key=operator.itemgetter(1))
    
    if ( (math.isnan(lat)) | (math.isnan(long)) ):
        return None, None
    else :
        return min_value, min_index+1

In [49]:
%%time
ads['Min_Distance'], ads['Marker_Location'] = zip(*ads.apply(lambda row: minimum_distance(row['LAT'], row['LONG'], row['Marker1_LAT'], row['Marker2_LAT'],
                                                            row['Marker3_LAT'], row['Marker4_LAT'], row['Marker5_LAT'], row['Marker6_LAT'],
                                                            row['Marker7_LAT'], row['Marker8_LAT'], row['Marker9_LAT'], row['Marker10_LAT'], 
                                                            row['Marker11_LAT'], row['Marker12_LAT'], row['Marker13_LAT'], row['Marker14_LAT'],
                                                            row['Marker15_LAT'], row['Marker16_LAT'], row['Marker17_LAT'], row['Marker18_LAT'],
                                                            row['Marker19_LAT'], row['Marker20_LAT'], row['Marker1_LONG'], row['Marker2_LONG'],
                                                            row['Marker3_LONG'], row['Marker4_LONG'], row['Marker5_LONG'], row['Marker6_LONG'], 
                                                            row['Marker7_LONG'], row['Marker8_LONG'], row['Marker9_LONG'], row['Marker10_LONG'],
                                                            row['Marker11_LONG'], row['Marker12_LONG'], row['Marker13_LONG'], row['Marker14_LONG'],
                                                            row['Marker15_LONG'], row['Marker16_LONG'], row['Marker17_LONG'], row['Marker18_LONG'], 
                                                            row['Marker19_LONG'], row['Marker20_LONG']),axis=1))

CPU times: user 2.94 s, sys: 0 ns, total: 2.94 s
Wall time: 2.94 s


In [50]:
ads = ads[['OUTAGE_ID', 'LAT', 'LONG', 'Min_Distance', 'Marker_Location']]
ads.head()

,OUTAGE_ID,LAT,LONG,Min_Distance,Marker_Location
0,2002645939,39.79,-86.19,3.63,11
1,2002645942,39.80,-85.99,3.27,13
2,2002645993,39.88,-86.02,4.16,5
3,2002645992,39.88,-86.01,3.55,5
4,2002645982,39.87,-86.04,4.11,4


In [51]:
ads['Marker_Location'] = 'Marker' + ads['Marker_Location'].astype(str)
print(ads.Marker_Location.unique())

['Marker11' 'Marker13' 'Marker5' 'Marker4' 'Marker8' 'Marker10' 'Marker15'
 'Marker2' 'Marker16' 'Marker7' 'Marker17' 'Marker12' 'Marker3' 'Marker9'
 'MarkerNone' 'Marker6' 'Marker20' 'Marker14' 'Marker18' 'Marker1'
 'Marker19']


In [52]:
ads.drop(['LAT','LONG'],axis=1,inplace=True)
ads.head()
print(df_ads.shape)
df_ads = pd.merge(df_ads, ads, how='left', on=['OUTAGE_ID'])
df_ads.drop_duplicates(keep='first', inplace=True)
print(df_ads.shape)

(13699, 79)
(13676, 81)


In [53]:
print(list(df_ads.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 

## **ADD CYCLICITY ACCORDING TO HOUR**

In [54]:
df_ads['Hour'] = df_ads['CREATION_DATETIME'].dt.hour
print(df_ads['Hour'].unique())

[ 0  3  8  9 10 11 12 13 14 16 17 20 23  4  5  7 15 18 21  1  2  6 19 22]


In [55]:
df_ads['Hour_Sin'] = np.sin(df_ads.Hour*(2.0*np.pi/24))
df_ads['Hour_Cos'] = np.cos(df_ads.Hour*(2.0*np.pi/24))

In [56]:
df_ads.drop(['Hour'],axis=1,inplace=True)

## **MAJ_OTG_ID ADDTION**
- Not valid for blue-sky + storm analytical dataset

In [57]:
# ads_final = pd.merge(df_ads, maj_otg_df, on=['INCIDENT_ID', 'STRCTUR_NO'], how='left')
# print(ads_final.shape)
# ads_final.head()

In [58]:
# create a copy of the previous dataframe so that rerunning of code can be avaoided
ads_final = df_ads.copy(deep=True)

## **ADD SUBSEQUENT OUTAGES**

In [59]:
ads_final['Date'] = ads_final.CREATION_DATETIME.dt.date
ads_final['RANK_SUBSEQUENT_OUTAGES'] = ads_final.groupby('Date')['CREATION_DATETIME'].rank(method='dense', ascending=True)
ads_final.drop(['Date','Min_Distance'],axis=1,inplace=True)

In [60]:
print(ads_final.shape)
ads_final.sort_values(by = ['CREATION_DATETIME'], inplace=True)
ads_final.head()

(13676, 83)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES
0,2002645939,2001465208.00,462-B/97,BLUE SKY,2020-01-01 00:03:00,2020-01-01 02:30:00,3209.00,FUSE,320.00,26,30.00,1.00,2020-01-01 03:45:00,30.00,100.00,100.00,0,147.00,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.79,-86.19,ZONE4,NaN,37,2,17,8,5,Marker11,0.00,1.00,1.00
1,2002645942,2001465216.00,481CB/195,BLUE SKY,2020-01-01 00:50:44,2020-01-01 01:15:40,2902.00,1TBOH,290.00,1,1.00,0.00,2020-01-01 04:30:00,1.00,0.00,0.00,0,24.93,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.80,-85.99,ZONE3,NaN,37,2,17,8,5,Marker13,0.00,1.00,2.00
2,2002645993,2001465223.00,280-B/81,BLUE SKY,2020-01-01 03:10:16,2020-01-01 04:18:28,3607.00,SWITCH,360.00,70,205.00,0.00,2020-01-01 05:45:00,205.00,2480.70,2480.70,0,68.20,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3,NaN,37,2,17,8,5,Marker5,0.71,0.71,3.00
3,2002645992,2001465223.00,281-B/4,BLUE SKY,2020-01-01 03:10:16,2020-01-01 05:23:41,3607.00,SWITCH,360.00,3,10.00,1.00,2020-01-01 08:00:00,10.00,152.00,152.00,0,133.42,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.01,ZONE3,NaN,37,2,17,8,5,Marker5,0.71,0.71,3.00
17,2002646057,2001465296.00,281-B/193,BLUE SKY,2020-01-01 03:10:16,2020-01-01 11:46:19,3607.00,FUSE,360.00,3,2.00,0.00,2020-01-01 07:45:00,2.00,25.00,25.00,0,516.05,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3,LOCATION CREATED BY OPENING DEVICE: 281-B/193 ...,37,2,17,8,5,Marker5,0.71,0.71,3.00


## **ADDING LIVE OUTAGE**

In [61]:
def count_outage(group):
    group = group.reset_index(drop = True)
    group['LIVE_OUTAGE'] = len(ads_final[(ads_final.CREATION_DATETIME < group.CREATION_DATETIME[0]) & (ads_final.ENERGIZED_DATETIME > group.CREATION_DATETIME[0])])
    return group

def grouping_fn(df):
    liveoutage = df.groupby(['OUTAGE_ID'], as_index=False).apply(count_outage)
    return liveoutage

if __name__ == '__main__':
    starttime = time.time()
    with Pool(30) as p:
            live_outage = p.map(grouping_fn, [ads_final[:5000], ads_final[5000:10000], ads_final[10000:15000],
                                  ads_final[15000:20000], ads_final[20000:25000], ads_final[25000:30000],
                                  ads_final[30000:35000], ads_final[35000:40000], ads_final[40000:50000],
                                  ads_final[50000:55000], ads_final[55000:60000], ads_final[60000:65000],
                                   ads_final[65000:70000], ads_final[70000:75000], ads_final[75000:80000],
                                  ads_final[80000:90000], ads_final[90000:100000], ads_final[100000:105000],
                                  ads_final[105000:]])
    print('That took {} seconds'.format(time.time() - starttime))

That took 22.01866888999939 seconds


In [62]:
ads_final=pd.concat(live_outage)

In [63]:
print(ads_final.shape)
ads_final.reset_index(drop=True,inplace=True)
ads_final.head()

(13676, 84)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES,LIVE_OUTAGE
0,2002645939,2001465208.00,462-B/97,BLUE SKY,2020-01-01 00:03:00,2020-01-01 02:30:00,3209.00,FUSE,320.00,26,30.00,1.00,2020-01-01 03:45:00,30.00,100.00,100.00,0,147.00,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.79,-86.19,ZONE4,NaN,37,2,17,8,5,Marker11,0.00,1.00,1.00,0
1,2002645942,2001465216.00,481CB/195,BLUE SKY,2020-01-01 00:50:44,2020-01-01 01:15:40,2902.00,1TBOH,290.00,1,1.00,0.00,2020-01-01 04:30:00,1.00,0.00,0.00,0,24.93,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.80,-85.99,ZONE3,NaN,37,2,17,8,5,Marker13,0.00,1.00,2.00,1
2,2002645982,2001465252.00,318--/116,BLUE SKY,2020-01-01 03:23:08,2020-01-01 05:09:12,3607.00,1TBOH,360.00,3,4.00,0.00,2020-01-01 07:00:00,4.00,25.00,25.00,0,106.07,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.87,-86.04,ZONE3,NaN,37,2,17,8,5,Marker4,0.71,0.71,5.00,13
3,2002645992,2001465223.00,281-B/4,BLUE SKY,2020-01-01 03:10:16,2020-01-01 05:23:41,3607.00,SWITCH,360.00,3,10.00,1.00,2020-01-01 08:00:00,10.00,152.00,152.00,0,133.42,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.01,ZONE3,NaN,37,2,17,8,5,Marker5,0.71,0.71,3.00,0
4,2002645993,2001465223.00,280-B/81,BLUE SKY,2020-01-01 03:10:16,2020-01-01 04:18:28,3607.00,SWITCH,360.00,70,205.00,0.00,2020-01-01 05:45:00,205.00,2480.70,2480.70,0,68.20,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3,NaN,37,2,17,8,5,Marker5,0.71,0.71,3.00,0


## **OUTAGE FEATURES**

In [64]:
def count_outage_minutes(group):
    group = group.reset_index(drop = True)
    df_temp = ads_final[['OUTAGE_ID','CREATION_DATETIME']]
    df_temp['minutes'] = (group['CREATION_DATETIME'][0] - ads_final['CREATION_DATETIME']).dt.total_seconds().div(60)
    df_temp = df_temp[df_temp.minutes > 0]
    group['Outages_in_last_1hr'] = len(df_temp[df_temp.minutes <= 60])
    group['Outages_in_last_2hr'] = len(df_temp[df_temp.minutes <= 120])
    group['Outages_in_last_3hr'] = len(df_temp[df_temp.minutes <= 180])
    group['Outages_in_last_4hr'] = len(df_temp[df_temp.minutes <= 240])
    group['Outages_in_last_5hr'] = len(df_temp[df_temp.minutes <= 300])
    group['Outages_in_last_6hr'] = len(df_temp[df_temp.minutes <= 360])
    group['Outages_in_last_7hr'] = len(df_temp[df_temp.minutes <= 420])
    group['Outages_in_last_8hr'] = len(df_temp[df_temp.minutes <= 480])
    group['Outages_in_last_9hr'] = len(df_temp[df_temp.minutes <= 540])
    group['Outages_in_last_10hr'] = len(df_temp[df_temp.minutes <= 600])
    return group

def grouping_fn_minutes(df):
    liveoutage = df.groupby(['OUTAGE_ID'], as_index=False).apply(count_outage_minutes)
    return liveoutage

if __name__ == '__main__':
    starttime = time.time()
    with Pool(30) as p:
            live_outage_minutes = p.map(grouping_fn_minutes, [ads_final[:5000], ads_final[5000:10000], ads_final[10000:15000],
                                  ads_final[15000:20000], ads_final[20000:25000], ads_final[25000:30000],
                                  ads_final[30000:35000], ads_final[35000:40000], ads_final[40000:50000],
                                  ads_final[50000:55000], ads_final[55000:60000], ads_final[60000:65000],
                                  ads_final[65000:70000], ads_final[70000:75000], ads_final[75000:80000],
                                  ads_final[80000:90000], ads_final[90000:100000], ads_final[100000:105000],
                                  ads_final[105000:]])
    print('That took {} seconds'.format(time.time() - starttime))

That took 91.08874988555908 seconds


In [65]:
ads_final=pd.concat(live_outage_minutes)

In [66]:
print(ads_final.shape)
ads_final.reset_index(drop=True,inplace=True)
ads_final.head()

(13676, 94)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES,LIVE_OUTAGE,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr
0,2002645939,2001465208.00,462-B/97,BLUE SKY,2020-01-01 00:03:00,2020-01-01 02:30:00,3209.00,FUSE,320.00,26,30.00,1.00,2020-01-01 03:45:00,30.00,100.00,100.00,0,147.00,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.79,-86.19,ZONE4,NaN,37,2,17,8,5,Marker11,0.00,1.00,1.00,0,0,0,0,0,0,0,0,0,0,0
1,2002645942,2001465216.00,481CB/195,BLUE SKY,2020-01-01 00:50:44,2020-01-01 01:15:40,2902.00,1TBOH,290.00,1,1.00,0.00,2020-01-01 04:30:00,1.00,0.00,0.00,0,24.93,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.80,-85.99,ZONE3,NaN,37,2,17,8,5,Marker13,0.00,1.00,2.00,1,1,1,1,1,1,1,1,1,1,1
2,2002645982,2001465252.00,318--/116,BLUE SKY,2020-01-01 03:23:08,2020-01-01 05:09:12,3607.00,1TBOH,360.00,3,4.00,0.00,2020-01-01 07:00:00,4.00,25.00,25.00,0,106.07,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.87,-86.04,ZONE3,NaN,37,2,17,8,5,Marker4,0.71,0.71,5.00,13,13,13,14,15,15,15,15,15,15,15
3,2002645992,2001465223.00,281-B/4,BLUE SKY,2020-01-01 03:10:16,2020-01-01 05:23:41,3607.00,SWITCH,360.00,3,10.00,1.00,2020-01-01 08:00:00,10.00,152.00,152.00,0,133.42,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.01,ZONE3,NaN,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2
4,2002645993,2001465223.00,280-B/81,BLUE SKY,2020-01-01 03:10:16,2020-01-01 04:18:28,3607.00,SWITCH,360.00,70,205.00,0.00,2020-01-01 05:45:00,205.00,2480.70,2480.70,0,68.20,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3,NaN,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2


In [67]:
df_copy = ads_final.copy(deep=True)
# ads_final = df_copy.copy(deep=True)

In [68]:
print(df_copy.shape)
print(ads_final.shape)

(13676, 94)
(13676, 94)


## **Day of the week features**

In [69]:
ads_final['Date'] = ads_final.CREATION_DATETIME.dt.date
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
ads_final['Weekday'] = ads_final['Date'].apply(lambda x: x.weekday()).apply(lambda x: days[x])

ads_final['Weekend_flag'] = ads_final['Weekday'].apply(lambda x: True if (x == 'Saturday') | (x == 'Sunday') else False)
ads_final.drop(['Date'],axis=1,inplace=True)

In [70]:
print(list(ads_final.columns))
print(ads_final.shape)

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 

## **Priority Queuing Feature**
1. Rank based on simple customer quantity as mentioned by Eric (live rankings to be followed, numerical feature) <br>

In [71]:
ads_final.sort_values(by = ['CREATION_DATETIME'], inplace=True)
ads_final.reset_index(drop=True, inplace=True)

In [72]:
def create_groups_based_on_live_outages(live):
    list_group_no = []
    group = 0
    
    for i in range(len(live)):
        if live.LIVE_OUTAGE[i] == 0  :
            group = group + 1
            list_group_no.append(group)
        else :
            list_group_no.append(group)
    
    return list_group_no

ads_final['Live_outage_group'] = create_groups_based_on_live_outages(ads_final)

In [73]:
ads_final['Priority_Customer_Qty'] = ads_final.groupby(['Live_outage_group'])['DOWNSTREAM_CUST_QTY'].rank(method='dense', ascending=False)

In [74]:
print(ads_final.shape)

(13676, 98)


2. Rank based on the factor of distance from centroid and customer quantity (live rankings to be followed, approach #2, numerical feature) <br>

In [75]:
df_v1 = ads_final.groupby(['Live_outage_group'],as_index=False).agg({'LAT' : 'sum', 'LONG' : 'sum', 'LIVE_OUTAGE' : 'count'})
df_v1['Center_LAT'] = (df_v1.LAT)/(df_v1.LIVE_OUTAGE)
df_v1['Center_LONG'] = (df_v1.LONG)/(df_v1.LIVE_OUTAGE)
df_v1.head()

,Live_outage_group,LAT,LONG,LIVE_OUTAGE,Center_LAT,Center_LONG
0,1,79.59,-172.18,2,39.79,-86.09
1,2,39.88,-86.02,1,39.88,-86.02
2,3,39.88,-86.01,1,39.88,-86.01
3,4,1154.89,-2496.21,29,39.82,-86.08
4,5,39.79,-86.08,1,39.79,-86.08


In [76]:
df_v1.drop(['LAT', 'LONG', 'LIVE_OUTAGE'], axis=1, inplace=True)
ads_final_v1 = pd.merge(ads_final, df_v1, how='left', on='Live_outage_group')

In [77]:
print(ads_final_v1.shape)
# ads_final_v1.head(5)

(13676, 100)


In [78]:
def cal_distance_from_center_lat_long(lat, long, center_lat, center_long):
    if ((math.isnan(lat)) | (math.isnan(long)) | (math.isnan(center_lat)) | (math.isnan(center_long))):
        return None
    else :
        coords1 = [lat,long]
        coords2 = [center_lat, center_long]
        return (geopy.distance.distance(coords1, coords2).miles)

In [79]:
ads_final_v1['Dis_From_Live_Centriod'] = ads_final_v1.apply(lambda x: cal_distance_from_center_lat_long(x['LAT'], x['LONG'], x['Center_LAT'], x['Center_LONG']),axis=1)
ads_final_v1['Dis_From_Live_Centriod'] = ads_final_v1['Dis_From_Live_Centriod'].apply(pd.to_numeric, errors='coerce')

In [80]:
ads_final_v1['Dis_From_Live_Centriod_div_Cust_qty'] = (ads_final_v1['Dis_From_Live_Centriod']) / (ads_final_v1['DOWNSTREAM_CUST_QTY'])
ads_final_v1['Priority_Dist_Customer_Qty'] = ads_final_v1.groupby(['Live_outage_group'])['Dis_From_Live_Centriod_div_Cust_qty'].rank(method='max', ascending=True)

In [81]:
ads_final_v1.drop(['Center_LAT', 'Center_LONG', 'Dis_From_Live_Centriod'], axis=1, inplace=True)

In [82]:
print(ads_final_v1.shape)

(13676, 100)


## **Add dispatch area location**

In [83]:
def cal_distance_from_dipatch_area(lat, long):
    
    if ((math.isnan(lat)) | (math.isnan(long))):
        return None, None
    else :
        coords1 = [lat,long]
        dist_34 = geopy.distance.distance(coords1, [39.8802, -86.2324]).miles
        dist_arl = geopy.distance.distance(coords1, [39.8802, -86.0854]).miles
        dist_mill = geopy.distance.distance(coords1, [39.7880, -86.2296]).miles
        dist_english = geopy.distance.distance(coords1, [39.7880, -86.0868]).miles
        dist_wii = geopy.distance.distance(coords1, [39.7003, -86.2303]).miles
        dist_south = geopy.distance.distance(coords1, [39.7003, -86.0834]).miles
    
        dist_list = [dist_34, dist_arl, dist_mill, dist_english, dist_wii, dist_south]

        min_index, min_value = min(enumerate(dist_list), key=operator.itemgetter(1))
    
        return min_value, min_index+1

In [84]:
ads_final_v1['Min_Distance'], ads_final_v1['Grid'] = zip(*ads_final_v1.apply(lambda row: cal_distance_from_dipatch_area(row['LAT'], row['LONG']),axis=1))

In [85]:
def map_grid_to_location(row):
    
    if row==1:
        return '34th'
    elif row==2:
        return 'ARL.'
    elif row==3:
        return 'MILL'
    elif row==4:
        return 'ENGLISH'
    elif row==5:
        return 'W.I.'
    elif row==6:
        return 'SOUTH'
    else :
        return "NO_LOCATION"

In [86]:
ads_final_v1['Dispatch_Location'] = ads_final_v1.apply(lambda row: map_grid_to_location(row['Grid']),axis=1)

In [87]:
ads_final_v1.drop(['Min_Distance', 'Grid'], axis=1, inplace=True)

In [88]:
ads_final_v1.head()

,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES,LIVE_OUTAGE,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr,Weekday,Weekend_flag,Live_outage_group,Priority_Customer_Qty,Dis_From_Live_Centriod_div_Cust_qty,Priority_Dist_Customer_Qty,Dispatch_Location
0,2002645939,2001465208.00,462-B/97,BLUE SKY,2020-01-01 00:03:00,2020-01-01 02:30:00,3209.00,FUSE,320.00,26,30.00,1.00,2020-01-01 03:45:00,30.00,100.00,100.00,0,147.00,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.79,-86.19,ZONE4,NaN,37,2,17,8,5,Marker11,0.00,1.00,1.00,0,0,0,0,0,0,0,0,0,0,0,Wednesday,False,1,1.00,0.18,1.00,MILL
1,2002645942,2001465216.00,481CB/195,BLUE SKY,2020-01-01 00:50:44,2020-01-01 01:15:40,2902.00,1TBOH,290.00,1,1.00,0.00,2020-01-01 04:30:00,1.00,0.00,0.00,0,24.93,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.80,-85.99,ZONE3,NaN,37,2,17,8,5,Marker13,0.00,1.00,2.00,1,1,1,1,1,1,1,1,1,1,1,Wednesday,False,1,2.00,5.39,2.00,ENGLISH
2,2002646057,2001465296.00,281-B/193,BLUE SKY,2020-01-01 03:10:16,2020-01-01 11:46:19,3607.00,FUSE,360.00,3,2.00,0.00,2020-01-01 07:45:00,2.00,25.00,25.00,0,516.05,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3,LOCATION CREATED BY OPENING DEVICE: 281-B/193 ...,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2,Wednesday,False,2,1.00,0.00,1.00,ARL.
3,2002645992,2001465223.00,281-B/4,BLUE SKY,2020-01-01 03:10:16,2020-01-01 05:23:41,3607.00,SWITCH,360.00,3,10.00,1.00,2020-01-01 08:00:00,10.00,152.00,152.00,0,133.42,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.01,ZONE3,NaN,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2,Wednesday,False,3,1.00,0.00,1.00,ARL.
4,2002645993,2001465223.00,280-B/81,BLUE SKY,2020-01-01 03:10:16,2020-01-01 04:18:28,3607.00,SWITCH,360.00,70,205.00,0.00,2020-01-01 05:45:00,205.00,2480.70,2480.70,0,68.20,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3,NaN,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2,Wednesday,False,4,2.00,0.02,1.00,ARL.


In [89]:
print(ads_final_v1.shape)
print(ads_final_v1.LAT.isnull().sum())
print(ads_final_v1.LONG.isnull().sum())

(13676, 101)
893
893


In [90]:
ads_final_v1.to_csv('gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_V10.4.2020_11132020.csv', index=False)

In [91]:
ads_final_v1['CREATION_DATETIME'] = pd.to_datetime(ads_final['CREATION_DATETIME'])
ads_final_v1['Date'] = ads_final_v1['CREATION_DATETIME'].dt.date
ads_final_v1['Date'] = ads_final_v1['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [92]:
ads_final_v1['Date'].nunique()

195

## **Weather data addition**

In [93]:
newdf_ws = pd.read_csv("gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Weather_Data_2020/IPL_Weather_Data2020.csv", index_col = False)

In [94]:
len(newdf_ws)

2502

In [95]:
columns = ['DAY_FLAG','POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG',
           'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG','OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
           'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG', 
           'WEATHER__FLOOD_CAUSE_FLG','STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG',
          'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG','TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG',
           'CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 'FOUND_OPEN_OCCURN_FLG', 'PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG', 
           'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG', 'CAP_BANK_OCCURN_FLG', 'RECLOSER_OCCURN_FLG', 'OH_OCCURN_FLG', 'PRIORITY_VAL_1.0', 'PRIORITY_VAL_2.0',
           'PRIORITY_VAL_3.0', 'PRIORITY_VAL_5.0']
for i in columns:
    ads_final_v1[i] = ads_final_v1[i].apply(lambda x: True if x==1 else False)

In [96]:
newdf_ws.dtypes

Unnamed: 0           int64
Marker_Location     object
Date                object
LAT                float64
LONG               float64
latitude           float64
longitude          float64
timestamp           object
cldCvrMin            int64
cldCvrAvg            int64
cldCvrMax            int64
dewPtMin           float64
dewPtAvg           float64
dewPtMax           float64
feelsLikeMin       float64
feelsLikeAvg       float64
feelsLikeMax       float64
heatIndexMin       float64
heatIndexAvg       float64
heatIndexMax       float64
mslPresMin         float64
mslPresAvg         float64
mslPresMax         float64
precip             float64
presTendMin        float64
presTendAvg        float64
presTendMax        float64
radSolarMin          int64
radSolarAvg        float64
radSolarMax        float64
radSolarTot        float64
relHumMin          float64
relHumAvg          float64
relHumMax          float64
sfcPresMin         float64
sfcPresAvg         float64
sfcPresMax         float64
s

In [97]:
newdf_ws['timestamp'] = pd.to_datetime(newdf_ws['timestamp'].str[:10]).dt.date.astype(str)
ads_final_v1['Date'] = pd.to_datetime(ads_final_v1['CREATION_DATETIME']).dt.date.astype(str)

In [98]:
newdf_ws = newdf_ws.drop( columns = ['Unnamed: 0'])

In [99]:
len(newdf_ws), len(newdf_ws.drop_duplicates())

(2502, 2502)

In [100]:
newdf_ws.head(2)

,Marker_Location,Date,LAT,LONG,latitude,longitude,timestamp,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,dewPtMax,feelsLikeMin,feelsLikeAvg,feelsLikeMax,heatIndexMin,heatIndexAvg,heatIndexMax,mslPresMin,mslPresAvg,mslPresMax,precip,presTendMin,presTendAvg,presTendMax,radSolarMin,radSolarAvg,radSolarMax,radSolarTot,relHumMin,relHumAvg,relHumMax,sfcPresMin,sfcPresAvg,sfcPresMax,snowDepth,snowfall,spcHumMin,spcHumAvg,spcHumMax,tempMin,tempAvg,tempMax,windChillMin,windChillAvg,windChillMax,windDirAvg,windDir80mAvg,windDir100mAvg,windSpdMin,windSpdAvg,windSpdMax,windSpd80mMin,windSpd80mAvg,windSpd80mMax,windSpd100mMin,windSpd100mAvg,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax
0,Marker11,2020-01-01 00:00:00+00:00,39.76,-86.20,39.76,-86.20,2020-01-01,0,27,100,25.50,27.90,29.80,26.20,33.40,42.80,31.80,38.60,46.70,1007.00,1010.70,1013.50,0.00,-1.00,-0.20,0.70,0,118.60,499.20,2846.90,43.20,67.00,89.00,981.60,984.80,987.40,0.00,0.00,3.00,3.30,3.50,31.80,38.60,46.70,26.20,33.40,42.80,220,216,218,5.00,7.20,10.40,12.50,18.00,28.60,13.00,17.90,27.70,30.50,34.70,39.30
1,Marker13,2020-01-01 00:00:00+00:00,39.77,-85.97,39.77,-85.97,2020-01-01,0,29,100,25.50,27.80,30.10,25.20,31.30,41.10,32.10,37.80,46.00,1007.20,1010.70,1013.50,0.00,-1.10,-0.20,0.60,0,117.50,502.50,2820.20,44.40,68.30,88.00,977.70,980.70,983.20,0.00,0.00,3.00,3.30,3.60,32.10,37.80,46.00,25.20,31.30,41.10,221,217,219,5.20,9.30,12.00,12.90,18.40,28.60,13.50,18.40,28.40,30.80,34.10,38.80


## **ADDING WEATHER FEATURES**

In [101]:
# removing unwanted columns

newdf_ws = newdf_ws.drop(['snowDepth'], axis = 1)
unwanted = newdf_ws.columns[newdf_ws.columns.str.startswith('presTend')]
newdf_ws = newdf_ws.drop(unwanted, axis=1)

In [102]:
newdf_ws.drop(['Date', 'LAT', 'LONG'], axis=1, inplace=True)

In [103]:
print(list(newdf_ws.columns))

['Marker_Location', 'latitude', 'longitude', 'timestamp', 'cldCvrMin', 'cldCvrAvg', 'cldCvrMax', 'dewPtMin', 'dewPtAvg', 'dewPtMax', 'feelsLikeMin', 'feelsLikeAvg', 'feelsLikeMax', 'heatIndexMin', 'heatIndexAvg', 'heatIndexMax', 'mslPresMin', 'mslPresAvg', 'mslPresMax', 'precip', 'radSolarMin', 'radSolarAvg', 'radSolarMax', 'radSolarTot', 'relHumMin', 'relHumAvg', 'relHumMax', 'sfcPresMin', 'sfcPresAvg', 'sfcPresMax', 'snowfall', 'spcHumMin', 'spcHumAvg', 'spcHumMax', 'tempMin', 'tempAvg', 'tempMax', 'windChillMin', 'windChillAvg', 'windChillMax', 'windDirAvg', 'windDir80mAvg', 'windDir100mAvg', 'windSpdMin', 'windSpdAvg', 'windSpdMax', 'windSpd80mMin', 'windSpd80mAvg', 'windSpd80mMax', 'windSpd100mMin', 'windSpd100mAvg', 'windSpd100mMax', 'wetBulbMin', 'wetBulbAvg', 'wetBulbMax']


In [104]:
## Add range for columns with negative values 

newdf_ws['tempRange'] = newdf_ws['tempMax'] - newdf_ws['tempMin']
newdf_ws['windSpdRange'] = newdf_ws['windSpdMax'] - newdf_ws['windSpdMin']
newdf_ws['sfcPresRange'] = newdf_ws['sfcPresMax'] - newdf_ws['sfcPresMin']
newdf_ws['cldCvrRange'] = newdf_ws['cldCvrMax'] - newdf_ws['cldCvrMin']
newdf_ws['relHumRange'] = newdf_ws['relHumMax'] - newdf_ws['relHumMin']

In [105]:
## Add ratio for columns which dont have negative values 

newdf_ws['relHumRatio'] = newdf_ws['relHumMax'] / newdf_ws['relHumMin']
newdf_ws['sfcPresRatio'] = newdf_ws['sfcPresMax'] / newdf_ws['sfcPresMin']

In [106]:
newdf_ws = newdf_ws.replace([np.inf, -np.inf], np.nan)
nulls = newdf_ws.isnull().sum()

df_nulls = pd.DataFrame({'Feature': nulls.index, 'VALUES': nulls.values})
df_nulls[df_nulls.VALUES>=1]

,Feature,VALUES


In [107]:
ads_df = pd.merge(ads_final_v1, newdf_ws,left_on = ['Date','Marker_Location'],right_on = ['timestamp','Marker_Location'],how = "left")

In [108]:
print(ads_df.shape[0])
print(len(ads_df.drop_duplicates()))

13676
13676


In [109]:
ads_df.drop_duplicates(keep='first', inplace=True)
ads_df.reset_index(drop=True,inplace=True)
print(ads_df.shape)

(13676, 163)


In [110]:
# removing unwanted columns for the final data frame

ads_df = ads_df.drop(['DOWNSTREAM_KVA_VAL','KVA_VAL','Date','latitude','longitude','timestamp', 'radSolarMin'], axis = 1)

In [113]:
print(list(ads_df.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCU

## **ADD ACTIVE NO OF TRUCKS PER DAY**

In [114]:
# read d16_memb_activity table from GCS landing bucket
d16_memb_activity = pd.read_csv(r'gs://aes-datahub-0001-landing/CAD/IPL/CAD.D16_MEMB_ACTIVITY.csv', sep = ";", encoding = "ISO-8859-1")
d16_memb_activity['Date'] = pd.to_datetime(d16_memb_activity['D16_DATETIME_KEY'].astype(str).str[:8])

# filter for DIST.OPS and DUTY.IN
# d16_memb_activity = d16_memb_activity[(d16_memb_activity.AGENCY_CODE == 'DIST.OPS') & (d16_memb_activity.D16S_CAD_ACTION == 'DUTY-IN')]
d16_memb_activity = d16_memb_activity[(d16_memb_activity.D16S_CAD_ACTION == 'DUTY-IN')]

active_crews = d16_memb_activity.groupby(['Date'],as_index=False).agg({'UNIT' : pd.Series.nunique})
active_crews.rename(columns={"UNIT":"Active_no_of_trucks_per_day"},inplace=True)

del d16_memb_activity

ads_df['Date'] = ads_df['CREATION_DATETIME'].dt.date
ads_df['Date'] = pd.to_datetime(ads_df['Date'])
active_crews['Date'] = pd.to_datetime(active_crews['Date'])

ads_df = pd.merge(ads_df, active_crews, how='left', on=['Date'])
ads_df.drop(['Date'], axis=1, inplace=True)
print(ads_df.shape)

(13676, 157)


In [115]:
ads_df = ads_df.drop_duplicates()
ads_df.reset_index(drop=True, inplace=True)
print(ads_df.shape)

(13676, 157)


In [116]:
ads_df.dropna(axis=0,subset=['LAT','LONG'], inplace=True)

In [117]:
ads_df.isnull().sum()

OUTAGE_ID                                  0
INCIDENT_ID                                0
STRCTUR_NO                                 0
EVENT                                      0
CREATION_DATETIME                          0
ENERGIZED_DATETIME                         0
CIRCT_ID                                   0
DNI_EQUIP_TYPE                             0
SUBST_ID                                   0
CALL_QTY                                   0
DOWNSTREAM_CUST_QTY                        0
KEY_CUST_QTY                               0
ETR_DATETIME                            2210
CUST_QTY                                   0
DAY_FLAG                                   0
TTR                                        0
POLE_CLUE_FLG                              0
PART_LIGHT_CLUE_FLG                        0
EMERGENCY_CLUE_FLG                         0
POWER_OUT_CLUE_FLG                         0
OPEN_DEVICE_CLUE_FLG                       0
TREE_CLUE_FLG                              0
WIRE_DOWN_

In [118]:
ads_df.dtypes

OUTAGE_ID                                       int64
INCIDENT_ID                                   float64
STRCTUR_NO                                     object
EVENT                                          object
CREATION_DATETIME                      datetime64[ns]
ENERGIZED_DATETIME                     datetime64[ns]
CIRCT_ID                                      float64
DNI_EQUIP_TYPE                                 object
SUBST_ID                                      float64
CALL_QTY                                        int64
DOWNSTREAM_CUST_QTY                           float64
KEY_CUST_QTY                                  float64
ETR_DATETIME                           datetime64[ns]
CUST_QTY                                      float64
DAY_FLAG                                         bool
TTR                                           float64
POLE_CLUE_FLG                                    bool
PART_LIGHT_CLUE_FLG                              bool
EMERGENCY_CLUE_FLG          

### **Drop columns which will not be used for modelling purpose**

In [119]:
ads_df = ads_df[ads_df['TTR'] >= 30]
ads_df.reset_index(drop=True, inplace=True)

In [120]:
print(ads_df.shape)
ads_df.head()

(8435, 157)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES,LIVE_OUTAGE,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr,Weekday,Weekend_flag,Live_outage_group,Priority_Customer_Qty,Dis_From_Live_Centriod_div_Cust_qty,Priority_Dist_Customer_Qty,Dispatch_Location,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,dewPtMax,feelsLikeMin,feelsLikeAvg,feelsLikeMax,heatIndexMin,heatIndexAvg,heatIndexMax,mslPresMin,mslPresAvg,mslPresMax,precip,radSolarAvg,radSolarMax,radSolarTot,relHumMin,relHumAvg,relHumMax,sfcPresMin,sfcPresAvg,sfcPresMax,snowfall,spcHumMin,spcHumAvg,spcHumMax,tempMin,tempAvg,tempMax,windChillMin,windChillAvg,windChillMax,windDirAvg,windDir80mAvg,windDir100mAvg,windSpdMin,windSpdAvg,windSpdMax,windSpd80mMin,windSpd80mAvg,windSpd80mMax,windSpd100mMin,windSpd100mAvg,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax,tempRange,windSpdRange,sfcPresRange,cldCvrRange,relHumRange,relHumRatio,sfcPresRatio,Active_no_of_trucks_per_day
0,2002645939,2001465208.00,462-B/97,BLUE SKY,2020-01-01 00:03:00,2020-01-01 02:30:00,3209.00,FUSE,320.00,26,30.00,1.00,2020-01-01 03:45:00,30.00,False,147.00,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,INDIANAPOLIS,39.79,-86.19,ZONE4,NaN,37,2,17,8,5,Marker11,0.00,1.00,1.00,0,0,0,0,0,0,0,0,0,0,0,Wednesday,False,1,1.00,0.18,1.00,MILL,0.00,27.00,100.00,25.50,27.90,29.80,26.20,33.40,42.80,31.80,38.60,46.70,1007.00,1010.70,1013.50,0.00,118.60,499.20,2846.90,43.20,67.00,89.00,981.60,984.80,987.40,0.00,3.00,3.30,3.50,31.80,38.60,46.70,26.20,33.40,42.80,220.00,216.00,218.00,5.00,7.20,10.40,12.50,18.00,28.60,13.00,17.90,27.70,30.50,34.70,39.30,14.90,5.40,5.80,100.00,45.80,2.06,1.01,nan
1,2002646057,2001465296.00,281-B/193,BLUE SKY,2020-01-01 03:10:16,2020-01-01 11:46:19,3607.00,FUSE,360.00,3,2.00,0.00,2020-01-01 07:45:00,2.00,False,516.05,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,INDIANAPOLIS,39.88,-86.02,ZONE3,LOCATION CREATED BY OPENING DEVICE: 281-B/193 ...,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2,Wednesday,False,2,1.00,0.00,1.00,ARL.,0.00,28.00

## **WRITE TO CSV**

In [123]:
ads_df.to_csv("gs://aes-analytics-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_2020_11162020.csv",index=False)

In [ ]:
ads_df.drop(['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'SUBST_ID', 
             'ETR_DATETIME', 'CUST_QTY', 'LAT', 'LONG', 'LIVE_OUTAGE', 'Live_outage_group', 'Marker_Location', 'Dis_From_Live_Centriod_div_Cust_qty',
             'Active_no_of_trucks_per_day'], axis=1, inplace=True)

In [ ]:
ads_df.to_csv("gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_V10.4.2020_COLUMNS.csv",index=False)